<a href="https://colab.research.google.com/github/ko-maki/code/blob/master/%E5%88%86%E9%A1%9E%E3%83%A2%E3%83%87%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

参考にしたサイトのURLは切れていました(2020年12月18日)

In [ ]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

In [ ]:
#テキストを読み込む。
#テキストは、ネットニュースの記事を用いる。
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!unzip "/content/gdrive/MyDrive/texts.zip"

In [ ]:
!ls texts/

In [ ]:
#以下、テキストの処理を行う。
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob

In [ ]:
#ディレクトリを取得する。
directories = glob('texts/*')
directories

In [ ]:
#ファイル名を取得する。
filepaths = glob('{}/*.txt'.format(directories[0]))
filepaths[:5]

In [ ]:
#ファイルを読み込む。
with open(filepaths[0], encoding='utf-8') as f:
  text = ''.join(f.readlines()[2:]) ## URL等の先頭２行を除く。
print(text)

In [ ]:
#ディレクトリの番号を取得する。
for (i, directory) in enumerate(directories):
    print(i, directory)

In [ ]:
#テキストを読み込み、ラベル付を行う。
texts, labels = [], []
for (i, directory) in enumerate(directories):
    #各ディレクトリ内のtxtファイルのパスをすべて取得
    filepaths = glob('{}/*.txt'.format(directory))
    # テキストを読み込んで、内容をtextに格納、ラベルも併せて格納
    for filepath in filepaths:
        with open(filepath, encoding='utf-8') as f:
            text = ''.join(f.readlines()[2:])  # URL等の先頭２行を除いた各行の文章を連結して格納
            texts.append(text)
            labels.append(i)

In [ ]:
#確認する。
len(texts), len(labels)

In [ ]:
texts[3850]

In [ ]:
labels[2849]

In [ ]:
#形態素解析を行う。
import MeCab
mecab = MeCab.Tagger('-Ochasen')

In [ ]:
#名詞を格納する。
def get_nouns(text):
    nouns = []
    res = mecab.parse(text)
    words = res.split('\n')[:-2]
    for word in words:
        part = word.split('\t')
        if '名詞' in part[3]:
            nouns.append(part[0])
    return nouns

In [ ]:
word_collect = []
for text in texts:
    nouns = get_nouns(text)
    word_collect.append(' '.join(nouns))

In [ ]:
#nouns

In [ ]:
#word_collect

In [ ]:
#分析を行う。
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=20)
x = vectorizer.fit_transform(word_collect)
x = x.toarray()

In [ ]:
#len(vectorizer.vocabulary_)

In [ ]:
#len(x)

In [ ]:
x = x.astype('float32')
t = np.array(labels, 'int32')

In [ ]:
#データを訓練用と検証用に分類する。
from sklearn.model_selection import train_test_split
x_train, x_test, t_train, t_test = train_test_split(x, t, train_size=0.7, random_state=0)

In [ ]:
#以下、モデルの定義を行う。
import tensorflow as tf

In [ ]:
import os
import random

def reset_seed(seed=0):
    
    os.environ['PYTHONHASHSEED'] = '0'
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

In [ ]:
_, n_input = x_train.shape
n_output = len(np.unique(t_test))

In [ ]:
from tensorflow.keras import models, layers

In [ ]:
reset_seed(0)

model = models.Sequential([
    layers.Dense(200, input_shape=(n_input, ), activation='relu'),
    layers.Dense(n_output, activation='softmax')
])

optimizer = tf.keras.optimizers.SGD(lr=0.01)
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer=optimizer, 
              metrics=['accuracy'])

In [ ]:
#モデルを学習させる。
history = model.fit(x_train, t_train,
          batch_size=100,
          epochs=20,
          verbose=1,
          validation_data=(x_test, t_test))

In [ ]:
# 学習結果を取得する。
results = pd.DataFrame(history.history)
results

In [ ]:
# 損失を可視化する。
results[['loss', 'val_loss']].plot(title='loss')
plt.xlabel('epochs')

In [ ]:
# 正解率を可視化する。
results[['accuracy', 'val_accuracy']].plot(title='metric')
plt.xlabel('epochs')